# Sentry alert data

Created: 2019-01-15

Last modified: 2019-01-15

# Methods

In [1]:
%matplotlib inline
import os
import sys
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.reset_orig()

/local/home/Asif/anaconda3/lib/python3.7/site-packages/matplotlib/__init__.py:855: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
/local/home/Asif/anaconda3/lib/python3.7/site-packages/matplotlib/__init__.py:846: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 2.2 and will be removed in 3.1.
  "2.2", name=key, obj_type="rcparam", addendum=addendum)


In [2]:
from textwrap import dedent
import findspark
import os
def _formulate_pyspark_submit_args(submit_args=None):
    pass
def _parse_master(pyspark_submit_args):
    sargs = pyspark_submit_args.split()
    for j,sarg in enumerate(sargs):
        if sarg == "--master":
            try:
                return sargs[j+1]
            except:
                raise Exception("Could not parse master from PYSPARK_SUBMIT_ARGS")
    raise Exception("Could not parse master from PYSPARK_SUBMIT_ARGS")
def initialize_spark(appName="MyApp",submit_args=None):
    """
    This function assumes you already have SPARK_HOME and PYSPARK_SUBMIT_ARGS environment variables set
    """
    if "SPARK_HOME" not in os.environ:
        raise Exception("SPARK_HOME environmental variable not set.")
    if 'PYSPARK_SUBMIT_ARGS' not in os.environ:
        os.environ['PYSPARK_SUBMIT_ARGS'] = '--master local[12] --driver-memory 8g --executor-memory 8g pyspark-shell'
    if "PYSPARK_SUBMIT_ARGS" not in os.environ:
        raise Exception(
                dedent("""\
                       PYSPARK_SUNBMIT_ARGS environmental variable not set.
                       
                       As an example:
                       export PYSPARK_SUBMIT_ARGS = " --master local[8] --driver-memory 8g --executor-memory 8g pyspark-shell"
                       """))
    findspark.init()
    spark_master = _parse_master(os.environ["PYSPARK_SUBMIT_ARGS"])
    from pyspark.sql import SparkSession
    spark = SparkSession\
                .builder\
                .master(spark_master)\
                .appName("MyApp")\
                .getOrCreate()
    return spark

spark = initialize_spark()

spark

In [3]:
import pyarrow.parquet as pq
from pyspark.sql import SQLContext
sq = SQLContext(spark)

In [18]:
EICU_PATH = '/home/p2017-999/acs_data/processed_data/eICU/eICU_parquet/'

In [49]:
!ls {EICU_PATH}

admissionDrug.parquet		   lab.parquet.very_old
admissionDx.parquet		   medication.parquet
allergy.parquet			   microLab.parquet
apacheApsVar.parquet		   note.parquet
apachePatientResults.parquet	   nurseAssessment.parquet
apachePatientResultsv2.parquet	   nurseCare.parquet
apachePredVar.parquet		   nurseCharting.parquet
autoAcuityOverrideHistory.parquet  pastHistory.parquet
autoAcuityQuickEntry.parquet	   patient.parquet
carePlanCareProvider.parquet	   physicalExam.parquet
carePlanEOL.parquet		   README
carePlanGeneral.parquet		   respiratoryCare.parquet
carePlanGoal.parquet		   respiratoryCharting.parquet
carePlanInfectiousDisease.parquet  rsReportBucket.parquet
customLab.parquet		   ssaDMF.parquet
diagnosis.parquet		   treatment.parquet
infusionDrug.parquet		   vitalAperiodic.parquet
intakeOutput.parquet		   vitalPeriodic.parquet
lab.parquet


In [19]:
def load_table_from_parquet(table_name, path=None, name=None):
    """Loads a table from its corresponding parquet files"""
    if path is None:
        path = EICU_PATH
    if name is None:
        name = table_name
    table = sq.read.load(f'{path}{table_name}.parquet')
    table.registerTempTable(name)
    return table

# find all the table names in the eICU directory
table_name_list = []
for item in os.listdir(EICU_PATH):
    if item.endswith('.parquet'):
        table_name_list.append(item[:-8])

print("The Number of Tables = ", len(table_name_list))
print("Table Name List: ")
print(table_name_list)

The Number of Tables =  35
Table Name List: 
['apachePatientResultsv2', 'admissionDx', 'allergy', 'apacheApsVar', 'apachePatientResults', 'apachePredVar', 'autoAcuityOverrideHistory', 'carePlanInfectiousDisease', 'autoAcuityQuickEntry', 'carePlanCareProvider', 'carePlanEOL', 'carePlanGeneral', 'carePlanGoal', 'customLab', 'diagnosis', 'infusionDrug', 'microLab', 'medication', 'nurseAssessment', 'nurseCare', 'nurseCharting', 'pastHistory', 'note', 'respiratoryCharting', 'rsReportBucket', 'ssaDMF', 'treatment', 'vitalAperiodic', 'lab', 'physicalExam', 'admissionDrug', 'vitalPeriodic', 'respiratoryCare', 'intakeOutput', 'patient']


In [20]:
admissionDx = load_table_from_parquet('admissionDx')
patient = load_table_from_parquet('patient')
apachePatientResults = load_table_from_parquet('apachePatientResults')

In [21]:
hospitalIDs = !ls {EICU_PATH}admissionDx.parquet

# Load data

In [4]:
PATH = '/home/p2017-999/acs_data/processed_data/eICU/Sentry/'
!ls {PATH}

Event  Final  Outcome  recombinedStayNew.csv  recombinedStayOld.csv


In [5]:
recombinedStayNew = pd.read_csv(os.path.join(PATH,'recombinedStayNew.csv'))

In [6]:
recombinedStayNew.shape

(2358086, 43)

In [7]:
recombinedStayNew.columns

Index(['TableName', 'dummyStayID', 'patientHealthSystemStayID',
       'patientUnitStayIDFirst', 'patientUnitStayIDLast', 'recombinedFlag',
       'gender', 'age', 'ethnicity', 'hospitalID', 'wardID',
       'apacheAdmissionDx', 'admissionHeight', 'hospitalAdmitYear',
       'hospitalAdmitTime', 'hospitalAdmitOffset', 'hospitalAdmitSource',
       'hospitalDischargeYear', 'hospitalDischargeTime',
       'hospitalDischargeOffset', 'hospitalDischargeLocation',
       'hospitalDischargeStatus', 'unitType', 'unitAdmitYear', 'unitAdmitTime',
       'unitAdmitOffset', 'unitAdmitSource', 'unitVisitNumber', 'unitStayType',
       'admissionWeight', 'dischargeWeight', 'unitDischargeYear',
       'unitDischargeTime', 'unitDischargeOffset', 'unitDischargeLocation',
       'unitDischargeStatus', 'cmoFlag', 'dnrFlag', 'readmitFlag',
       'timeToReadmission', 'icuDischargeInternal',
       'dischargeLocationMissingID', 'drsFlag'],
      dtype='object')

In [48]:
recombinedStayNew.patientHealthSystemStayID.nunique()

2219534

In [47]:
recombinedStayNew[recombinedStayNew.recombinedFlag==1].patientHealthSystemStayID.nunique()

67457

In [8]:
# unique hospital ids
hospitalids = recombinedStayNew.hospitalID.unique()

In [9]:
recombinedStayNew.head()

,TableName,dummyStayID,patientHealthSystemStayID,patientUnitStayIDFirst,patientUnitStayIDLast,recombinedFlag,gender,age,ethnicity,hospitalID,...,unitDischargeOffset,unitDischargeLocation,unitDischargeStatus,cmoFlag,dnrFlag,readmitFlag,timeToReadmission,icuDischargeInternal,dischargeLocationMissingID,drsFlag
0,recombinedStay,1,128894,141136,141136,0,Female,81,Caucasian,73,...,17744,Floor,Alive,0,0,0,NaN,1,NaN,0
1,recombinedStay,2,128905,141151,141151,0,Male,67,Caucasian,56,...,4352,Home,Alive,0,0,0,NaN,0,NaN,0
2,recombinedStay,3,128919,141168,141168,0,Female,70,Caucasian,59,...,3596,Death,Expired,0,0,0,NaN,0,NaN,1
3,recombinedStay,4,128921,141170,141170,0,Male,75,Caucasian,73,...,8310,Floor,Alive,0,1,0,NaN,1,NaN,0
4,recombinedStay,5,128931,141183,141183,0,Female,29,African American,73,...,4014,Home,Alive,0,0,0,NaN,0,NaN,1


In [12]:
# duplicate health system ids
k = recombinedStayNew.duplicated(['patientHealthSystemStayID'])
dids = recombinedStayNew.loc[k].patientHealthSystemStayID.unique()

In [13]:
recombinedStayNew.loc[recombinedStayNew.patientHealthSystemStayID == dids[3]]

,TableName,dummyStayID,patientHealthSystemStayID,patientUnitStayIDFirst,patientUnitStayIDLast,recombinedFlag,gender,age,ethnicity,hospitalID,...,unitDischargeOffset,unitDischargeLocation,unitDischargeStatus,cmoFlag,dnrFlag,readmitFlag,timeToReadmission,icuDischargeInternal,dischargeLocationMissingID,drsFlag
49,recombinedStay,50,128902,141146,141146,0,Male,86,Caucasian,71,...,41849,Floor,Alive,0,1,0,NaN,1,NaN,1
65,recombinedStay,66,128902,141145,141145,0,Male,86,Caucasian,71,...,10004,Floor,Alive,0,0,0,NaN,1,NaN,1


In [15]:
recombinedStayNew.loc[recombinedStayNew.patientUnitStayIDFirst != recombinedStayNew.patientUnitStayIDLast].head()

,TableName,dummyStayID,patientHealthSystemStayID,patientUnitStayIDFirst,patientUnitStayIDLast,recombinedFlag,gender,age,ethnicity,hospitalID,...,unitDischargeOffset,unitDischargeLocation,unitDischargeStatus,cmoFlag,dnrFlag,readmitFlag,timeToReadmission,icuDischargeInternal,dischargeLocationMissingID,drsFlag
164,recombinedStay,165,129039,141313,141314,1,Male,45,Caucasian,73,...,1078,Death,Expired,0,0,0,NaN,0,NaN,0
224,recombinedStay,225,129174,141473,141472,1,Female,49,Hispanic,73,...,21301,Floor,Alive,0,0,0,NaN,1,NaN,0
321,recombinedStay,322,129280,141609,141612,1,Male,75,Caucasian,73,...,8575,Floor,Alive,0,0,1,5804.0,1,NaN,0
322,recombinedStay,323,129290,141623,141626,1,Male,62,Caucasian,60,...,4721,stepdown/other,Alive,0,0,0,NaN,1,NaN,1
329,recombinedStay,330,129349,141710,141712,1,Female,44,Caucasian,63,...,1501,stepdown/other,Alive,0,1,0,NaN,1,NaN,0


In [16]:
recombinedStayNew.loc[:,['patientHealthSystemStayID','patientUnitStayIDFirst','patientUnitStayIDLast','recombinedFlag','hospitalID']]

,patientHealthSystemStayID,patientUnitStayIDFirst,patientUnitStayIDLast,recombinedFlag,hospitalID
0,128894,141136,141136,0,73
1,128905,141151,141151,0,56
2,128919,141168,141168,0,59
3,128921,141170,141170,0,73
4,128931,141183,141183,0,73
5,128933,141185,141185,0,69
6,128946,141200,141200,0,71
7,128947,141202,141202,0,63
8,128960,141217,141217,0,66
9,128961,141219,141219,0,73


# Query eICU

In [33]:
recombinedStayNew[recombinedStayNew.patientHealthSystemStayID == 2743055]

,TableName,dummyStayID,patientHealthSystemStayID,patientUnitStayIDFirst,patientUnitStayIDLast,recombinedFlag,gender,age,ethnicity,hospitalID,...,unitDischargeOffset,unitDischargeLocation,unitDischargeStatus,cmoFlag,dnrFlag,readmitFlag,timeToReadmission,icuDischargeInternal,dischargeLocationMissingID,drsFlag
2357765,recombinedStay,2357766,2743055,3353198,3353198,0,Female,66,Caucasian,458,...,91573,Step-Down Unit (SDU),Alive,0,0,1,42318.0,1,NaN,0
2357766,recombinedStay,2357767,2743055,3353200,3353200,0,Female,66,Caucasian,458,...,140374,Step-Down Unit (SDU),Alive,0,0,0,NaN,1,NaN,0
2357814,recombinedStay,2357815,2743055,3353196,3353196,0,Female,66,Caucasian,458,...,54272,Step-Down Unit (SDU),Alive,0,0,1,3765.0,1,NaN,0
2357823,recombinedStay,2357824,2743055,3353201,3353201,0,Female,66,Caucasian,458,...,63862,Step-Down Unit (SDU),Alive,0,0,1,4680.0,1,NaN,0
2358077,recombinedStay,2358078,2743055,3353197,3353199,1,Female,66,Caucasian,458,...,52859,Step-Down Unit (SDU),Alive,0,0,1,182.0,1,NaN,0


In [39]:
q = '''
select * from patient where patientHealthSystemStayID=2743055 and hospitalID=458
'''
res = sq.sql(q).toPandas()

In [40]:
res.columns

Index(['patientUnitStayID', 'patientHealthSystemStayID', 'gender', 'age',
       'ethnicity', 'wardID', 'apacheAdmissionDx', 'admissionHeight',
       'hospitalAdmitYear', 'hospitalAdmitTime24', 'hospitalAdmitTime',
       'hospitalAdmitOffset', 'hospitalAdmitSource', 'hospitalDischargeYear',
       'hospitalDischargeTime24', 'hospitalDischargeTime',
       'hospitalDischargeOffset', 'hospitalDischargeLocation',
       'hospitalDischargeStatus', 'unitType', 'unitAdmitYear',
       'unitAdmitTime24', 'unitAdmitTime', 'unitAdmitSource',
       'unitVisitNumber', 'unitStayType', 'admissionWeight', 'dischargeWeight',
       'unitDischargeYear', 'unitDischargeTime24', 'unitDischargeTime',
       'unitDischargeOffset', 'unitDischargeLocation', 'unitDischargeStatus',
       'auditEtlRunID', 'uniquePID', 'hospitalID'],
      dtype='object')

In [46]:
res.loc[:,['patientUnitStayID','hospitalAdmitOffset','unitVisitNumber','unitDischargeOffset','unitType']]

,patientUnitStayID,hospitalAdmitOffset,unitVisitNumber,unitDischargeOffset,unitType
0,3353196,-53041,3,1231,Cardiac ICU
1,3353197,-12441,1,29871,CTICU
2,3353198,-68542,5,23031,MICU
3,3353199,-42312,2,10547,Cardiac ICU
4,3353200,-133891,6,6483,Cardiac ICU
5,3353201,-58037,4,5825,Cardiac ICU
